### Imports

In [ ]:
import math
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Constants

In [ ]:
PATH_FILE_ORDER = './file/olist_orders_dataset.csv'
PATH_FILE_ORDER_ITEM = './file/olist_order_items_dataset.csv'
PATH_FILE_ORDER_REVIEW = './file/olist_order_reviews_dataset.csv'
PATH_FILE_ORDER_PAYMENT = './file/olist_order_payments_dataset.csv'

PATH_FILE_PROD = './file/olist_products_dataset.csv'
PATH_FILE_SELLER = './file/olist_sellers_dataset.csv'
PATH_FILE_CUSTOMER = './file/olist_customers_dataset.csv'
PATH_FILE_GEOLOCATION = './file/olist_geolocation_dataset.csv'
PATH_FILE_PROD_CATEGORY_TRANSLATE = './file/product_category_name_translation.csv'

# Reviews

### Constants

In [ ]:

# Original columns
COL_REV_ID = 'review_id'
COL_REV_MSG = 'review_comment_message'
COL_REV_ORDER = 'order_id'
COL_REV_SCORE = 'review_score'
COL_REV_TITLE = 'review_comment_title'
COL_REV_ANSWER = 'review_answer_timestamp'
COL_REV_CREATION = 'review_creation_date'

# Custom Columns
COL_REV_CUS_MSG_LEN = 'Message Length'

### Build Dataframe

In [ ]:
# Import file
reviewsDF = pd.read_csv(PATH_FILE_ORDER_REVIEW)
reviewsDFClean = reviewsDF[reviewsDF[COL_REV_MSG].notnull()]

# Handle NaN values
reviewsDF.loc[reviewsDF[COL_REV_MSG].isnull(), COL_REV_MSG] = ''
reviewsDF.loc[reviewsDF[COL_REV_TITLE].isnull(), COL_REV_TITLE] = ''

reviewsDFClean.loc[reviewsDFClean[COL_REV_TITLE].isnull(), COL_REV_TITLE] = ''

# Compute review lengths
reviewsDF[COL_REV_CUS_MSG_LEN] = reviewsDF[COL_REV_MSG].apply(lambda msg: len(msg))
reviewsDFClean[COL_REV_CUS_MSG_LEN] = reviewsDFClean[COL_REV_MSG].apply(lambda msg: len(msg))

# Format
sort_review = [COL_REV_SCORE, COL_REV_CUS_MSG_LEN, COL_REV_CREATION, COL_REV_MSG, COL_REV_TITLE]
# columns_review = [COL_REV_SCORE, COL_REV_CUS_MSG_LEN, COL_REV_MSG, COL_REV_TITLE]
reviewsDF = reviewsDF.sort_values(by=sort_review, ascending=False)
reviewsDFClean = reviewsDFClean.sort_values(by=sort_review, ascending=False)


### Statistics

In [ ]:
reviewsCount = reviewsDF.shape[0]
noMsgReviewsCount = reviewsDF.loc[reviewsDF[COL_REV_CUS_MSG_LEN] == 0].shape[0]
noMsgReviewsRatio = round(noMsgReviewsCount / reviewsCount * 100, 2)

print(f'Reviews: {reviewsCount}')
print(f'Empty message reviews: {noMsgReviewsCount} / {reviewsCount} ({noMsgReviewsRatio}%)')

scoreValues = sorted(reviewsDF[COL_REV_SCORE].unique())
for score in scoreValues:

    scoredDF = reviewsDF.loc[reviewsDF[COL_REV_SCORE] == score]

    scoredCount = scoredDF.shape[0]
    scoredRatio = round(scoredCount / reviewsCount * 100, 2)
    noMsgScoredCount = scoredDF.loc[reviewsDF[COL_REV_CUS_MSG_LEN] == 0].shape[0]
    noMsgScoredRation = round(noMsgScoredCount / scoredCount * 100, 2)

    print(f'{score} Score: {scoredCount} / {reviewsCount} ({scoredRatio}%)')
    print(f'\tNo message: {noMsgScoredCount} / {scoredCount} ({noMsgScoredRation}%)')

### Plots

In [ ]:

# Build grid
figure = plt.figure(figsize=(26, 8))

a11 = plt.subplot2grid((2, 4), (0, 0), fig=figure)
a12 = plt.subplot2grid((2, 4), (0, 1), fig=figure)
a13 = plt.subplot2grid((2, 4), (0, 2), fig=figure)

a14 = plt.subplot2grid((2, 4), (0, 3), fig=figure, rowspan=2)

a21 = plt.subplot2grid((2, 4), (1, 0), fig=figure)
a22 = plt.subplot2grid((2, 4), (1, 1), fig=figure)
a23 = plt.subplot2grid((2, 4), (1, 2), fig=figure)

# Bars graph: Total of review per each score.
bars = pd.DataFrame({
        'reviews': [
            reviewsDF[reviewsDF[COL_REV_SCORE] == 1].shape[0],
            reviewsDF[reviewsDF[COL_REV_SCORE] == 2].shape[0],
            reviewsDF[reviewsDF[COL_REV_SCORE] == 3].shape[0],
            reviewsDF[reviewsDF[COL_REV_SCORE] == 4].shape[0],
            reviewsDF[reviewsDF[COL_REV_SCORE] == 5].shape[0],
        ]
    },
    index=[1, 2, 3, 4, 5],
)

bars.plot.bar(ax=a14, title='Review Scores', color='cyan')

# Bars graph: Show proportion of reviews with or without comments per each score level.
yes = []
no = []
scores = []

for i in range(1, 6):
    _yes = reviewsDF[(reviewsDF[COL_REV_SCORE] == i) & (reviewsDF[COL_REV_CUS_MSG_LEN] > 0)].shape[0]
    _no = reviewsDF[(reviewsDF[COL_REV_SCORE] == i) & (reviewsDF[COL_REV_CUS_MSG_LEN] == 0)].shape[0]
    total = _yes + _no

    scores.append('Score: 0' + str(i))
    yes.append(_yes)
    no.append(_no)

barh = pd.DataFrame({ 'Yes': yes, 'No': no }, index=scores)
barh.plot.barh(
    ax=a13,
    title='Comments proportion by each score',
    color={ 'Yes': 'green', 'No': 'orange', 'AVG': 'c'},
)

# Histograms: Show length of commentaries per each review score level
figPositionMap = {
    1: a11, 2: a12,
    3: a21, 4: a22, 5: a23,
}

for i in range(1, 6):
    a = figPositionMap.get(i)
    a.hist(reviewsDFClean[reviewsDFClean[COL_REV_SCORE] == i][COL_REV_CUS_MSG_LEN].values, bins=15, facecolor='y', snap=False)
    a.set_title('Score ' + str(i))

figure.suptitle('Comment X Reviews')
figure.show()

# Orders

### Constants

In [ ]:

# Original columns: Orders
COL_ORD_ID = 'order_id'
COL_ORD_STATUS = 'order_status'
COL_ORD_CUSTOMER = 'customer_id'

COL_ORD_DATE_BUY = 'order_purchase_timestamp'
COL_ORD_DATE_APPROV = 'order_approved_at'
COL_ORD_DATE_CARRIER = 'order_delivered_carrier_date'
COL_ORD_DATE_DELIVER = 'order_delivered_customer_date'
COL_ORD_DATE_DELIVER_EST = 'order_estimated_delivery_date'

# Original columns: Order Items
COL_ORD_ITEM_ID = 'order_item_id'
COL_ORD_ITEM_PROD = 'product_id'
COL_ORD_ITEM_PRICE = 'price'
COL_ORD_ITEM_ORDER = 'order_id'
COL_ORD_ITEM_SELLER = 'seller_id'
COL_ORD_ITEM_FREIGHT = 'freight_value'
COL_ORD_ITEM_DATE_SHIP_LIMIT = 'shipping_limit_date'

# Custom columns
COL_ORD_CUS_TIME_DELAY = 'Delivery Delay'
COL_ORD_CUS_TIME_DELIVER = 'Time to Deliver'
COL_ORD_CUS_PRICE = 'Order Price'
COL_ORD_CUS_FREIGHT = 'Order Freight'

# Status
STATUS_ORD_APPROV = 'approved'
STATUS_ORD_CANCEL = 'canceled'
STATUS_ORD_CREATED = 'created'
STATUS_ORD_DELIVERED = 'delivered'
STATUS_ORD_INVOICED = 'invoiced'
STATUS_ORD_PROCESSING = 'processing'
STATUS_ORD_SHIPPED = 'shipped'
STATUS_ORD_UNAVAILABLE = 'unavailable'

# Rename
sortOrder = [COL_ORD_STATUS, COL_ORD_CUS_TIME_DELAY, COL_ORD_CUS_TIME_DELIVER, COL_ORD_CUS_FREIGHT, COL_ORD_CUS_PRICE]

columnsOrder = [
    COL_ORD_STATUS,
    COL_ORD_DATE_BUY, COL_ORD_DATE_DELIVER_EST, COL_ORD_DATE_DELIVER,
    COL_ORD_CUS_TIME_DELIVER, COL_ORD_CUS_TIME_DELAY,
    COL_ORD_CUS_PRICE, COL_ORD_CUS_FREIGHT
]


### Utils

In [ ]:

def getDaysInterval(mainDate: datetime.date, relativeDate: datetime.date = None, isRelativeToToday = True):
    '''
        TODO: 2021-10-14 - ADD Description
    '''

    upperDate = datetime.datetime.today() if isRelativeToToday else relativeDate
    return math.floor((upperDate - mainDate) / np.timedelta64(1,'D'))



def getDaysDelay(maxDate: datetime.date, deliveryDate: np.datetime64 = None):
    '''
        TODO: 2021-10-14 - ADD Description
    '''

    upperDate = deliveryDate or datetime.datetime.today()
    isDelayed = upperDate > maxDate
    return getDaysInterval(deliveryDate, upperDate, False) if isDelayed else 0

def setDeliveryDays(df: pd.DataFrame) -> pd.Series:
    '''
        TODO: 2021-10-14 - ADD Description
    '''
    
    return np.vectorize(getDaysInterval)(
        df[COL_ORD_DATE_DELIVER_EST],
        df[COL_ORD_DATE_DELIVER],
        df[COL_ORD_STATUS] != STATUS_ORD_DELIVERED,
    )


def setDelayDays(df: pd.DataFrame) -> pd.Series:
    '''
        TODO: 2021-10-14 - ADD Description
    '''
    
    return np.vectorize(getDaysDelay)(
        df[COL_ORD_DATE_DELIVER_EST],
        df[COL_ORD_DATE_DELIVER],
    )

def getOrderPrice(source: pd.Series, orderID: str) -> float:
    '''
        TODO: 2021-10-15 - ADD Description
    '''
    
    try:
        return source.loc[orderID]
    except KeyError:
        return 0

### Build Dataframe

In [ ]:

# Import files
dateColumns = [COL_ORD_DATE_BUY, COL_ORD_DATE_DELIVER, COL_ORD_DATE_DELIVER_EST]

ordersDF = pd.read_csv(PATH_FILE_ORDER, parse_dates=dateColumns)
orderItemDF = pd.read_csv(PATH_FILE_ORDER_ITEM)

# Clean data: Step 01 (remove inconsistent & unavailable data)
ordersDF = ordersDF[
  (ordersDF[COL_ORD_STATUS] != STATUS_ORD_UNAVAILABLE)
  & ~((ordersDF[COL_ORD_STATUS] == STATUS_ORD_DELIVERED) & (ordersDF[COL_ORD_DATE_DELIVER].isnull())) # Avoid failure on time interval calculations
]

# Add calculated interval fields 
ordersDF[COL_ORD_CUS_TIME_DELAY] = setDelayDays(ordersDF)
ordersDF[COL_ORD_CUS_TIME_DELIVER] = setDeliveryDays(ordersDF)

# Clean data: Step 02 (keep only orders unfinished or finished with consistent data)
endStatuses = [STATUS_ORD_CANCEL, STATUS_ORD_DELIVERED, STATUS_ORD_UNAVAILABLE]
midStatuses = [STATUS_ORD_CREATED, STATUS_ORD_APPROV, STATUS_ORD_INVOICED, STATUS_ORD_PROCESSING, STATUS_ORD_SHIPPED]

ordersDF = ordersDF[
    (ordersDF[COL_ORD_CUS_TIME_DELAY] >= 0) | (ordersDF[COL_ORD_CUS_TIME_DELIVER] >= 0)
    | ~ordersDF[COL_ORD_STATUS].isin(endStatuses)
]

# Add order price & freight price
# productMeanPrice = orderItemDF[ [COL_ORD_ITEM_PROD, COL_ORD_ITEM_PRICE] ].groupby(COL_ORD_ITEM_PROD).mean().sort_values(by=COL_ORD_ITEM_PROD)
orderFreight = orderItemDF[ [COL_ORD_ITEM_ORDER, COL_ORD_ITEM_FREIGHT] ].groupby(COL_ORD_ITEM_ORDER).sum().sort_values(by=COL_ORD_ITEM_ORDER)
orderPrice = orderItemDF[ [COL_ORD_ITEM_ORDER, COL_ORD_ITEM_PRICE] ].groupby(COL_ORD_ITEM_ORDER).sum().sort_values(by=COL_ORD_ITEM_ORDER)

ordersDF[COL_ORD_CUS_FREIGHT] = ordersDF[COL_ORD_ID].apply(lambda id: getOrderPrice(orderFreight[COL_ORD_ITEM_FREIGHT], id))
ordersDF[COL_ORD_CUS_PRICE] = ordersDF[COL_ORD_ID].apply(lambda id: getOrderPrice(orderPrice[COL_ORD_ITEM_PRICE], id))

ordersDF = ordersDF.sort_values(by=sortOrder, ascending=True, na_position='first')